In [1]:
import numpy as np

In [2]:
mesh_width = 10  # количество ячеек по пространству
mesh_height = 5  # количество ячеек по времени

tau = 10  # сут, временной шаг
h = 100  # м, длина ячейки

In [3]:
p_initial = 19e6  # Па, начальное давление в ячейках
k = 5  # мД, проницаемость
mu = 5  # мПа*с, вязкость жидкости
m = 0.15  # пористость
c = 2.4 * 10 ** (-9)  # Па^(-1), общая сжимаемость

q_left = 6  # м3/сут, расход на левой границе
p_right = 20e6  # Па, давление на правой границе
a = 10000  # м2, площадь поверхности боковой грани ячейки

In [4]:
p_values = np.zeros([mesh_height, mesh_width])
p_values[0] = [p_initial] * mesh_width
p_values

array([[19000000., 19000000., 19000000., 19000000., 19000000., 19000000.,
        19000000., 19000000., 19000000., 19000000.],
       [       0.,        0.,        0.,        0.,        0.,        0.,
               0.,        0.,        0.,        0.],
       [       0.,        0.,        0.,        0.,        0.,        0.,
               0.,        0.,        0.,        0.],
       [       0.,        0.,        0.,        0.,        0.,        0.,
               0.,        0.,        0.,        0.],
       [       0.,        0.,        0.,        0.,        0.,        0.,
               0.,        0.,        0.,        0.]])

In [5]:
def explicit_scheme():
    for time_step in range(1, mesh_height):
        p_values[time_step][0] = p_values[time_step - 1][0] + \
        (tau / h ** 2) * (k / (m * mu * c)) * (p_values[time_step - 1][1] - p_values[time_step - 1][0] + q_left * mu / (a * k) * h)
        p_values[time_step][-1] = p_values[time_step - 1][-1] + \
        4 / 3 * (tau / h ** 2) * (k / (m * mu * c)) * (2 * p_right - 3 * p_values[time_step - 1][-1] + p_values[time_step - 1][-2])
        
        for space_step in range(1, mesh_width - 1):
            p_values[time_step][space_step] = p_values[time_step - 1][space_step] + \
            (tau / h ** 2) * (k / (m * mu * c)) * (p_values[time_step - 1][space_step + 1] - 2 * p_values[time_step - 1][space_step] + p_values[time_step - 1][space_step - 1])

In [6]:
explicit_scheme()

In [7]:
p_values

array([[ 1.90000000e+07,  1.90000000e+07,  1.90000000e+07,
         1.90000000e+07,  1.90000000e+07,  1.90000000e+07,
         1.90000000e+07,  1.90000000e+07,  1.90000000e+07,
         1.90000000e+07],
       [ 1.91666667e+07,  1.90000000e+07,  1.90000000e+07,
         1.90000000e+07,  1.90000000e+07,  1.90000000e+07,
         1.90000000e+07,  1.90000000e+07,  1.90000000e+07,
         7.40742641e+12],
       [-4.62943630e+11,  4.62981963e+11,  1.90000000e+07,
         1.90000000e+07,  1.90000000e+07,  1.90000000e+07,
         1.90000000e+07,  1.90000000e+07,  2.05761317e+19,
        -8.23045119e+19],
       [ 2.57201507e+18, -3.85802330e+18,  1.28600823e+18,
         1.90000000e+07,  1.90000000e+07,  1.90000000e+07,
         1.90000000e+07,  5.71559214e+25, -3.42935466e+26,
         9.90702390e+26],
       [-1.78612151e+25,  3.21501903e+25, -1.78612203e+25,
         3.57224508e+24,  1.90000000e+07,  1.90000000e+07,
         1.58766448e+32, -1.27013136e+33,  4.81591422e+33,
        -1.